# Building inverted index and answering queries

In the first part of the lab you are going to implement a standard document processing pipeline and then build a simple search engine based on it:
- starting from crawling documents, 
- then building an inverted index,
- and answering queries using this index.

## Preprocessing

First, we need a unified approach to documents and queries preprocessing. Implement a class responsible for that. Complete the code for given functions (most of them are just one-liners) and make sure you pass the tests. Make use of `nltk` library, `spacy`, or any other you know.

In [ ]:
import nltk

class Preprocessor:
    
    def __init__(self):
        self.stop_words = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
                      'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with'}
        self.ps = nltk.stem.PorterStemmer()

    
    def tokenize(self, text):
        #TODO word tokenize text using nltk lib
        return ['one', 'two', 'three']

    
    def stem(self, word, stemmer):
        #TODO stem word using provided stemmer
        return 'stemmed_word'

    
    def is_apt_word(self, word):
        #TODO check if word is appropriate - not a stop word and isalpha, 
        # i.e consists of letters, not punctuation, numbers, dates
        return False

    
    def preprocess(self, text):
        #TODO combine all previous methods together: tokenize lowercased text 
        # and stem it, ignoring not appropriate words
        return ['one', 'two', 'three']

### Tests

In [ ]:
prep = Preprocessor()
text = 'To be, or not to be, that is the question'

assert prep.tokenize(text) == ['To', 'be', ',', 'or', 'not', 'to', 'be', ',', 'that', 'is', 'the', 'question']
assert prep.stem('retrieval', prep.ps) == 'retriev'
assert prep.is_apt_word('qwerty123') is False
assert prep.preprocess(text) == ['or', 'not', 'question']

## Crawling and Indexing

### Base classes

Here are some base classes you will need for writing an indexer. The code is given, still, you need to change some. Namely, the `parse` method (it is also possible to use your own implementations of other methods, just make sure they work). The reason to change is that the method always makes complete parsing, which we want to avoid, when we only need e.g. links or a specific portions of text.

In [ ]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse
import os


class Document:

    def __init__(self, url):
        self.url = url

    def download(self):
        try:
            response = requests.get(self.url)
            if response.status_code == 200:
                self.content = response.content
                return True
            else:
                return False
        except:
            return False

    def persist(self, path):
        with open(os.path.join(path, quote(self.url).replace('/', '_')), 'wb') as f:
            f.write(self.content)


class HtmlDocument(Document):

    def normalize(self, href):
        if href is not None and href[:4] != 'http':
            href = urllib.parse.urljoin(self.url, href)
        return href

    def parse(self):
        #TODO change this method
        
        def tag_visible(element):
            if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
                return False
            if isinstance(element, Comment):
                return False
            return True
   
        model = BeautifulSoup(self.content)
        
        self.anchors = []
        a = model.find_all('a')
        for anchor in a:
            href = self.normalize(anchor.get('href'))
            text = anchor.text
            self.anchors.append((text, href))
                        
        self.images = []
        i = model.find_all('img')
        for img in i:
            href = self.normalize(img.get('src'))
            self.images.append(href)
        
        texts = model.findAll(text=True)
        visible_texts = filter(tag_visible, texts)  
        self.text = u" ".join(t.strip() for t in visible_texts)

### Main class

The main indexer logic is here. We organize it as a crawler generator that adds certain visited pages to inverted index and saves them on disk. 

- `crawl_generator_for_index` method crawles the given website doing BFS, starting from `source` within given `depth`. Considers only inner pages (starting with https://www.reuters.com/...) for visiting. To speed up, do not consider pages with content type other than `html`: `.pdf`, `.mp3`, `.avi`, `.mp4`, `.txt`, ... . If crawler encounters an article page (of a form https://www.reuters.com/article/...), it saves its content in a file in `collection_path` folder and populates the inverted index calling `index_doc` method. When done, saves on disk three resulting dictionaries:
    - `doc_urls`: `{doc_id : url}`
    - `index`: `{term : [collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]}`
    - `doc_lengths`: `{doc_id : doc_length}` 

    `limit` parameter is given for testing - if not `None`, break the loop when number of saved articles exceeds the `limit` and return without writing dictionaries to disk.
    
- `index_doc` method parses and preprocesses the content of a `doc` and adds it to the inverted index. Also keeps track of document lengths in a `doc_lengths` dictionary.

Your crawler have to print visited urls as it runs.

In [ ]:
from collections import Counter
from queue import Queue
import pickle
import os

class Indexer:

    def __init__(self):      
        # dictionaries to populate
        self.doc_urls = {}        
        self.index = {}
        self.doc_lengths = {}
        # preprocessor
        self.prep = Preprocessor()
        
    
    def crawl_generator_for_index(self, source, depth, collection_path="collection", limit=None):        
        #TODO generate url-s for visiting
        
        
        # save results for later use
        with open('doc_urls.p', 'wb') as fp:
            pickle.dump(self.doc_urls, fp, protocol=pickle.HIGHEST_PROTOCOL)
        with open('inverted_index.p', 'wb') as fp:
            pickle.dump(self.index, fp, protocol=pickle.HIGHEST_PROTOCOL)
        with open('doc_lengths.p', 'wb') as fp:
            pickle.dump(self.doc_lengths, fp, protocol=pickle.HIGHEST_PROTOCOL)
    
        
    def index_doc(self, doc, doc_id):
        #TODO add documents to index
        pass

### Tests

Please make sure your crawler prints out urls with `print(k, c.url)`

In [ ]:
indexer = Indexer()
k = 1
for c in indexer.crawl_generator_for_index("https://www.reuters.com/technology", 2, "test_collection", 5):
    print(k, c.url)
    k += 1

assert type(indexer.index) is dict
assert type(indexer.index['reuter']) is list
assert type(indexer.index['reuter'][0]) is int
assert type(indexer.index['reuter'][1]) is tuple

### 1.2.4. Building an index

In [ ]:
indexer = Indexer()
k = 1
for c in indexer.crawl_generator_for_index("https://www.reuters.com/", 3, "docs_collection"):
    print(k, c.url)
    k += 1

### Index statistics

Load the index and print the statistics.

In [ ]:
# load index, doc_lengths and doc_urls
with open('inverted_index.p', 'rb') as fp:
    index = pickle.load(fp)
with open('doc_lengths.p', 'rb') as fp:
    doc_lengths = pickle.load(fp)
with open('doc_urls.p', 'rb') as fp:
    doc_urls = pickle.load(fp)
    
    
print('Total index length', len(index))
print('\nTop terms by number of documents they apperared in:')
sorted_by_n_docs = sorted(index.items(), key=lambda kv: (len(kv[1]), kv[0]), reverse=True)
print([(sorted_by_n_docs[i][0], len(sorted_by_n_docs[i][1])) for i in range(20)])
print('\nTop terms by overall frequency:')
sorted_by_freq = sorted(index.items(), key=lambda kv: (kv[1][0], kv[0]), reverse=True)
print([(sorted_by_freq[i][0], sorted_by_freq[i][1][0]) for i in range(20)])

## Answering a query (finally)

Now, given that we already have built the inverted index, it's time to utilize it for answering user queries. In this class there are two methods you need to implement:
- `boolean_retrieval`, the simplest form of document retrieval which returns a set of documents such that each one contains all query terms. Returns a set of document ids. Refer to *ch.1* of the book for details;
- `okapi_scoring`, Okapi BM25 ranking function - assigns scores to documents in the collection that are relevant to the user query. Returns a dictionary of scores, `doc_id:score`. Read about it in [Wikipedia](https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function) and implement accordingly.

Both methods accept `query` parameter in a form of a dictionary, `term:frequency`

In [ ]:
from collections import Counter
import math

class QueryProcessing:
    
    @staticmethod
    def prepare_query(raw_query):
        prep = Preprocessor()
        # pre-process query the same way as documents
        query = prep.preprocess(raw_query)
        # count frequency
        return Counter(query)
    
    @staticmethod
    def boolean_retrieval(query, index):
        #TODO retrieve a set of documents containing all query terms
        return {0, 1, 3}

    
    @staticmethod
    def okapi_scoring(query, doc_lengths, index, k1=1.2, b=0.75):
        #TODO retrieve relevant documents with scores
        return {0: 0.32, 5: 1.17}

### Tests

In [ ]:
test_doc_lengths = {1: 20, 2: 15, 3: 10, 4:20, 5:30}
test_index = {'x': [2, (1, 1), (2, 1)], 'y': [2, (1, 1), (3, 1)], 'z': [3, (2, 1), (4,2)]}


test_query1 = QueryProcessing.prepare_query('x z')
test_query2 = QueryProcessing.prepare_query('x y')


assert QueryProcessing.boolean_retrieval(test_query1, test_index) == {2}
assert QueryProcessing.boolean_retrieval(test_query2, test_index) == {1}
okapi_res = QueryProcessing.okapi_scoring(test_query2, test_doc_lengths, test_index)
assert all(k in okapi_res for k in (1, 2, 3))
assert not any(k in okapi_res for k in (4, 5))
assert okapi_res[1] > okapi_res[3] > okapi_res[2]

### And now query the real index

In [ ]:
import time

queries = ["russia ukraine", "Hong Kong", "new cases omicron"]
for q in queries:
    print(q)
    qobj = QueryProcessing.prepare_query(q)
    for res in QueryProcessing.boolean_retrieval(qobj, index):
        print('\t', doc_urls[res])

    s = time.time()
    okapi_res = QueryProcessing.okapi_scoring(qobj, doc_lengths, index)
    e = time.time()
    print(f"\t == Okapi Time: {e - s:.5f} ==")
    for res in okapi_res:
        print('\t', doc_urls[res], okapi_res[res])
    
    print()